In [ ]:
library(tidyverse)
library(dplyr)
library(tidyr)

In [ ]:
BiocManager::install("Biobase")

In [ ]:
library(Biobase)

In [ ]:
ptemp <- read.csv("../input/aml-fab/GSE147515_FAB_lbl.csv")
head(ptemp)

In [ ]:
ptemp$X.1 <- NULL
#ptemp <- ptemp[ptemp$FAB %in% c("CTRL", "M0"),]

In [ ]:
rownames(ptemp) <- ptemp$X
ptemp[1] <- NULL
head(ptemp)

In [ ]:
pdata <- AnnotatedDataFrame(ptemp)

In [ ]:
etemp <- read.csv("../input/aml-fab/GSE147515_FAB.csv")

In [ ]:
head(etemp)

In [ ]:
etemp$X <- NULL
#etemp <- etemp %>% select(ID, row.names(ptemp))

In [ ]:
rownames(etemp) <- etemp$ID
etemp[1] <- NULL
head(etemp)

In [ ]:
exprs <- as.matrix(etemp)

In [ ]:
eset <- new("ExpressionSet", exprs=exprs, phenoData=pdata)

# PCA

In [ ]:
library(ggplot2)
library(ggrepel)

In [ ]:
pca <- prcomp(t(exprs(eset)))

cbind(pData(eset), pca$x) %>%
ggplot(aes(x=PC1, y=PC2, col=FAB)) + geom_point()

# DGE

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+pData(eset)$FAB)
head(design)

In [ ]:
colnames(design) <- c("CTRL", "M0", "M1", "M2", "M3", "M4", "M5")

In [ ]:
head(exprs(eset))

In [ ]:
## calculate median expression level
cutoff <- median(exprs(eset))

## TRUE or FALSE for whether each gene is "expressed" in each sample
is_expressed <- exprs(eset) > cutoff

## Identify genes expressed in more than 2 samples

keep <- rowSums(is_expressed) > 2

## check how many genes are removed / retained.
table(keep)

## subset to just those expressed genes
eset <- eset[keep,]

In [ ]:
fit <- lmFit(exprs(eset), design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M0 - CTRL,
                           levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
full_results <- topTable(fit2, number=Inf)
full_results <- tibble::rownames_to_column(full_results,"ID")

In [ ]:
head(full_results)

In [ ]:
#mart <- useMart("ENSEMBL_MART_ENSEMBL", host="uswest.ensembl.org")

In [ ]:
#mart <- useDataset("hsapiens_gene_ensembl", mart)

In [ ]:
annotLookup <- getBM(
    mart=mart,
    attributes=c(
        "affy_hg_u133_plus_2",
        "ensembl_gene_id",
        "entrezgene_id"
    ),
    filter="affy_hg_u133_plus_2",
    values = full_results$ID,
    uniqueRows=TRUE
)

In [ ]:
annotLookup <- read.csv("C:/Users/neelj/Documents/HIVE/FAB Feature Selecion/aml-fab/annotLookup.csv")

In [ ]:
head(annotLookup)

In [ ]:
indicesLookup <- match(full_results$ID, annotLookup$affy_hg_u133_plus_2)

In [ ]:
full_results$entrezgene_id <- annotLookup[indicesLookup, "entrezgene_id"]
head(full_results)

In [ ]:
library(ggplot2)

In [ ]:
ggplot(full_results, aes(x=logFC, y=B)) + geom_point()

In [ ]:
p_cutoff <- 0.05
fc_cutoff <- 2

full_results %>% 
  mutate(Significant = adj.P.Val < p_cutoff, abs(logFC) > fc_cutoff ) %>% 
  ggplot(aes(x = logFC, y = B, col=Significant)) + geom_point()

In [ ]:
p_cutoff <- 0.05
fc_cutoff <- 1
topN <- 30

full_results %>% 
  mutate(Significant = adj.P.Val < p_cutoff, abs(logFC) > fc_cutoff ) %>% 
  mutate(Rank = 1:n(), Label = ifelse(Rank < topN, entrezgene_id,"")) %>% 
  ggplot(aes(x = logFC, y = B, col=Significant,label=Label)) + geom_point() + geom_text_repel(col="black")

In [ ]:
write.csv(full_results, "C:/Users/neelj/Documents/HIVE/FAB Feature Selecion/dge-data/m0/dge_results.csv")

In [ ]:
ids_of_interest <- mutate(full_results, Rank = 1:n()) %>%
    filter(Rank < topN) %>%
    pull(ID)

In [ ]:
gene_names <- mutate(full_results, Rank = 1:n()) %>%
    filter(Rank < topN) %>%
    pull(entrezgene_id)

In [ ]:
gene_matrix <- exprs(eset)[ids_of_interest,]

In [ ]:
library(pheatmap)

In [ ]:
# pheatmap(gene_matrix, labels_row = gene_names)

In [ ]:
write.csv(ids_of_interest, "C:/Users/neelj/Documents/HIVE/FAB Feature Selecion/dge-data/m0/top.csv")

In [ ]:
ids_of_interest

# Pathway Analysis

In [ ]:
BiocManager::install(version="3.12")

In [ ]:
if(!"rWikiPathways" %in% installed.packages()){
    if (!requireNamespace("BiocManager", quietly = TRUE))
        install.packages("BiocManager")
    BiocManager::install("rWikiPathways", update = FALSE)
}
library(rWikiPathways)


In [ ]:
load.libs <- c(
  "DOSE",
  "GO.db",
  "GSEABase",
  "org.Hs.eg.db",
  "clusterProfiler",
  "dplyr",
  "tidyr",
  "ggplot2",
  "stringr",
  "RColorBrewer",
  "rWikiPathways",
  "RCy3")
options(install.packages.check.source = "no")
options(install.packages.compile.from.source = "never")
if (!require("pacman")) install.packages("pacman"); library(pacman)
p_load(load.libs, update = TRUE, character.only = TRUE)
status <- sapply(load.libs,require,character.only = TRUE)
if(all(status)){
    print("SUCCESS: You have successfully installed and loaded all required libraries.")
} else{
    cat("ERROR: One or more libraries failed to install correctly. Check the following list for FALSE cases and try again...\n\n")
    status
}

In [ ]:
BiocManager::install("clusterProfiler")


In [ ]:
library(clusterProfiler)

In [ ]:
cytoscapePing()

In [ ]:
installApp('WikiPathways') 
installApp('CyTargetLinker') 
installApp('stringApp') 
installApp('enrichmentMap')

In [ ]:
#install.packages("tidyr", dependencies=TRUE)

In [ ]:
up.genes.entrez <- full_results[full_results$logFC > 1 & full_results$adj.P.Val < 0.05, "entrezgene_id"]
dn.genes.entrez <- full_results[full_results$logFC < -1 & full_results$adj.P.Val < 0.05, "entrezgene_id"]
bkgd.genes.entrez <- full_results[, "entrezgene_id"]

### Upreg

In [ ]:
egobp <- clusterProfiler::enrichGO(
        gene     = up.genes.entrez,
        universe = as.character(bkgd.genes.entrez),
        OrgDb    = org.Hs.eg.db,
        ont      = "BP",
        pAdjustMethod = "fdr",
        pvalueCutoff = 0.05, 
        readable = TRUE)


In [ ]:
head(egobp)

In [ ]:
barplot(egobp, showCategory = 20)

In [ ]:
dotplot(egobp, showCategory = 20)

In [ ]:
emapplot(egobp, showCategory = 20)

In [ ]:
goplot(egobp)

In [ ]:
# wp.hs.gmt <- rWikiPathways::downloadPathwayArchive(organism="Homo sapiens", format = "gmt")

wp.hs.gmt <- "wikipathways-20210810-gmt-Homo_sapiens.gmt"

In [ ]:
listOrganisms()

In [ ]:
wp2gene <- rWikiPathways::readPathwayGMT(wp.hs.gmt)

# Test

In [ ]:
lung.expr <- read.csv(system.file("extdata","data-lung-cancer.csv", package="rWikiPathways"),stringsAsFactors = FALSE)
nrow(lung.expr)
head(lung.expr)

In [ ]:
up.genes <- lung.expr[lung.expr$log2FC > 1 & lung.expr$adj.P.Value < 0.05, 1] 
dn.genes <- lung.expr[lung.expr$log2FC < -1 & lung.expr$adj.P.Value < 0.05, 1]
bkgd.genes <- lung.expr[,1]

In [ ]:
up.genes.entrez <- clusterProfiler::bitr(up.genes,fromType = "ENSEMBL",toType = "ENTREZID",OrgDb = org.Hs.eg.db)
cat("\n\nWhich column contains my new Entrez IDs?\n")
head(up.genes.entrez)


In [ ]:
up.genes.entrez[[2]]